## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.6666,-52.7314,62.76,90,90,8.05,overcast clouds
1,1,Mount Isa,AU,-20.7333,139.5000,51.57,37,4,4.61,clear sky
2,2,Along,IN,28.1667,94.7667,77.61,99,100,0.89,overcast clouds
3,3,Belaya Gora,RU,68.5333,146.4167,46.49,68,91,7.56,overcast clouds
4,4,De-Kastri,RU,51.4667,140.7833,65.19,90,100,5.21,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Along,IN,28.1667,94.7667,77.61,99,100,0.89,overcast clouds
11,11,Colombo,LK,6.9319,79.8478,80.55,88,86,12.24,light rain
12,12,Valentin Gomez Farias,MX,28.2167,-106.5833,74.43,47,48,4.09,scattered clouds
14,14,Atuona,PF,-9.8000,-139.0333,78.33,65,50,23.58,scattered clouds
15,15,Hilo,US,19.7297,-155.0900,74.80,97,90,5.75,moderate rain
19,19,Fernley,US,39.6080,-119.2518,84.74,15,74,5.82,broken clouds
21,21,Avarua,CK,-21.2078,-159.7750,71.65,100,90,5.75,overcast clouds
22,22,Yambio,SS,4.5721,28.3955,72.52,88,71,3.09,light rain
27,27,Albany,US,42.6001,-73.9662,73.98,84,97,5.41,overcast clouds
31,31,Bengkulu,ID,-3.8004,102.2655,76.93,89,3,4.79,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                387
City                   387
Country                386
Lat                    387
Lng                    387
Max Temp               387
Humidity               387
Cloudiness             387
Wind Speed             387
Current Description    387
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                386
City                   386
Country                386
Lat                    386
Lng                    386
Max Temp               386
Humidity               386
Cloudiness             386
Wind Speed             386
Current Description    386
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Along,IN,77.61,overcast clouds,28.1667,94.7667,
11,Colombo,LK,80.55,light rain,6.9319,79.8478,
12,Valentin Gomez Farias,MX,74.43,scattered clouds,28.2167,-106.5833,
14,Atuona,PF,78.33,scattered clouds,-9.8000,-139.0333,
15,Hilo,US,74.80,moderate rain,19.7297,-155.0900,
19,Fernley,US,84.74,broken clouds,39.6080,-119.2518,
21,Avarua,CK,71.65,overcast clouds,-21.2078,-159.7750,
22,Yambio,SS,72.52,light rain,4.5721,28.3955,
27,Albany,US,73.98,overcast clouds,42.6001,-73.9662,
31,Bengkulu,ID,76.93,clear sky,-3.8004,102.2655,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 386 entries, 2 to 682
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 386 non-null    object 
 1   Country              386 non-null    object 
 2   Max Temp             386 non-null    float64
 3   Current Description  386 non-null    object 
 4   Lat                  386 non-null    float64
 5   Lng                  386 non-null    float64
 6   Hotel Name           386 non-null    object 
dtypes: float64(3), object(4)
memory usage: 32.2+ KB


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.count()

City                   386
Country                386
Max Temp               386
Current Description    386
Lat                    386
Lng                    386
Hotel Name             386
dtype: int64

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))